# Llamastack Eval 🦙

We would like to be able to evaluate our models and applications before they go into production.  
To do that we can use the llamastack eval endpoint 🙌  
It allows us to run prompts and expected answers through different evaluations to make sure that the model answers as we expect.  
The prompts and expected answers can either be some you custom add, or it can be taken from an evaluation dataset such as this one from HuggingFace: https://huggingface.co/datasets/llamastack/simpleqa  

We will be testing two types of evaluations here:
- "subset_of" which tests if the LLM output is an exact subset of the expected answer 
- "llm_as_judge" which lets an LLM evaluate how similar the LLM output is to the expected answer

In here we will both test the raw model to see how performant it is, as well as the backend endpoints as well so that we also evaluate how effective our system prompts are. We will se even more on evaluating the raw model in a later chapter 😉

## Set-up
Let's start by installing the llamastack client and pointing it to our llamastack server

In [1]:
!pip install -q git+https://github.com/meta-llama/llama-stack.git@release-0.2.12 rich


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from llama_stack_client import LlamaStackClient
from rich.pretty import Pretty

base_url = "http://llama-stack.user1-test.svc.cluster.local:80"
client = LlamaStackClient(
    base_url=base_url,
    timeout=600.0 # Default is 1 min which is far too little for some agentic tests, we set it to 10 min
)

# Llama Stack Eval endpoint

Before we evaluate our backend endpoint, let's just try out llamastack eval and see how it works.  
First, we can see what scoring functions that Llama Stack Eval provides out-of-the-box.

In [15]:
Pretty(client.scoring_functions.list())

[
    ScoringFn(
        identifier='basic::equality',
        metadata={},
        provider_id='basic',
        return_type=ReturnType(type='number'),
        type='scoring_function',
        description='Returns 1.0 if the input is equal to the target, 0.0 otherwise.',
        params=BasicScoringFnParams(aggregation_functions=['accuracy'], type='basic'),
        provider_resource_id='equality'
    ),
    ScoringFn(
        identifier='basic::subset_of',
        metadata={},
        provider_id='basic',
        return_type=ReturnType(type='number'),
        type='scoring_function',
        description='Returns 1.0 if the expected is included in generated, 0.0 otherwise.',
        params=BasicScoringFnParams(aggregation_functions=['accuracy'], type='basic'),
        provider_resource_id='subset-of'
    ),
    ScoringFn(
        identifier='basic::regex_parser_multiple_choice_answer',
        metadata={},
        provider_id='basic',
        return_type=ReturnType(type='number'),
        type='scoring_function',
        description='Extract answer from response matching Answer: [the_answer_letter], and compare with expected 
result',
        params=RegexParserScoringFnParams(
            aggregation_functions=['accuracy'],
            parsing_regexes=[
                '(?i)The best answer is \\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)Answer\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)Answer\\s*:\u200b\u200b\u200b\u200b\u200b\u200b\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]
|[Ｄ])',
                '(?i)উত্তর\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)उत्तर\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)উত্তরঃ\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)উত্তর\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)Antwort\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)답변\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)정답\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)답\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)答案\\s*：\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)答案\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)答\\s*：\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)答\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)答复\\s*：\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)答曰\\s*：\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)الإجابة:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)الجواب:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)إجابة:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)الإجابة النهائية:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)الإجابة الصحيحة:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)الإجابة الصحيحة هي:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)الإجابة هي:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)Respuesta\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)Risposta\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)答え\\s*:\\s*([A-D]|[أ-د]|[অ]|[ব]|[ড]|[ঢ]|[Ａ]|[Ｂ]|[Ｃ]|[Ｄ])',
                '(?i)答え\\s*：\\s*([A-D]|[أ-د]

## Subset Of

Let's start by setting the `scoring_params` to use the `subset_of` function mentioned earlier and see what it comes back with.  
Here we create some `handmade_eval_rows` with the input and the expected answer, but we also add the generated answer already filled out.  

In [4]:
handmade_eval_rows = [
    {
        "input_query": "What is your favorite food?",
        "generated_answer": "Tapas are my favorites.",
        "expected_answer": "Tapas",
    },
    {
        "input_query": "What is your favorite food?",
        "generated_answer": "I really like tapas.",
        "expected_answer": "Tapas",
    }
]
Pretty(handmade_eval_rows)

scoring_params = {
    "basic::subset_of": None,
}
scoring_response = client.scoring.score(
    input_rows=handmade_eval_rows, scoring_functions=scoring_params
)
Pretty(scoring_response)

INFO:httpx:HTTP Request: POST http://llama-stack.user1-test.svc.cluster.local/v1/scoring/score "HTTP/1.1 200 OK"


ScoringScoreResponse(
    results={
        'basic::subset_of': ScoringResult(
            aggregated_results={'accuracy': {'accuracy': 0.5, 'num_correct': 1.0, 'num_total': 2}},
            score_rows=[{'score': 1.0}, {'score': 0.0}]
        )
    }
)

Hmm, we got half of the answers correct 🤔  
This is because we expect tapas to be spelled with a big T in front. As mentioned before, the subset_of function expects exact matches within the generated answer.

## Start Simple

Not only is it much faster to start with simple functional or integration tests, but it also allows us to keep the cost down as we don't evaluate the text in the response from the LLM as much as just evaluate some parameters around it or the effect it has.  
This can test things such as: 
- **Response metadata**: latency, token count, or response length - we will see more of this in a little bit.
- **Structural checks**: ensuring the output is valid JSON, fits a schema, or contains required fields.
- **Behavioral signals**: whether the model calls the right API, triggers the expected downstream function, or returns a specific keyword.

Llama Stack eval is a bit limited in what evaluations it currently provides, but we can create custom ones that help us evaluate these smaller, simpler tests.  
We have a summary usecase, so two that would fit ours is simply:  
- Is the response shorter than the input?
- Does the response get parsed in the app?

We'll cover the first one here just to showcase how this could look like.

https://llama-stack.readthedocs.io/en/latest/references/api_reference/index.html#/paths/v1-scoring-functions/post

In [ ]:
"""
Custom eval function that uses Llama Stack helper functions
"""
from llama_stack_client.types import ScoringScoreResponse
from llama_stack.apis.scoring import ScoringResult

def is_shorter(input, response):
    return {"score": int(response < input)}

def eval_is_shorter(input_rows: list[dict]):
    score_rows = []
    aggregated_results = {}
    for row in input_rows:
        score_rows.append(
            is_shorter(
                input=row["input_query"], 
                response=row["generated_answer"]
                )
            )

    num_total = len(score_rows)
    num_correct = sum(row["score"] for row in score_rows)
    accuracy = (num_correct / num_total) if num_total else 0.0

    aggregated_results = {
        "accuracy": {
            "accuracy": accuracy,
            "num_correct": float(num_correct),
            "num_total": num_total,
        }
    }
    
    results = {
        "is_shorter::custom": ScoringResult(score_rows=score_rows, aggregated_results=aggregated_results).model_dump()
    }
    return ScoringScoreResponse(results=results)

In [17]:
scoring_response = eval_is_shorter(
    input_rows = handmade_eval_rows
)
Pretty(scoring_response)

ScoringScoreResponse(
    results={
        'is_shorter::custom': ScoringResult(
            aggregated_results={'accuracy': {'accuracy': 1.0, 'num_correct': 2.0, 'num_total': 2}},
            score_rows=[{'score': 1.0}, {'score': 1.0}]
        )
    }
)

## LLM as judge🧑‍⚖️

Now let's try the same thing but with LLM as judge.   
Here we are going to feed our eval_rows into an LLM to evaluate how well the generated answer matches the expected answer.  
To make sure that the judge LLM does this, we also provide it a `JUDGE_PROMPT` it should follow, as well as a regex of expected scores (`judge_score_regexes`) from the judge.  
In our case we let it grade the generated answers from A to E (no F's in this class 🙂‍↔️), each with its own meaning that you can see in the judge prompt.  
We also choose Llama 3.2 to be our judge (as we all know llamas to be the best of judges). This means that when we later later evaluate replies from the backend, we will use the same LLM to generate our answer and judge them, essentially doing a self-judge strategy. This is not always the best, but works pretty well with Llama 3.2 and we don't have any other model to use right now.

In [8]:
handmade_eval_rows = [
    {
        "input_query": "What is your favorite food?",
        "generated_answer": "Tapas are my favorites.",
        "expected_answer": "Tapas",
    },
    {
        "input_query": "What is your favorite food?",
        "generated_answer": "I really like tapas.",
        "expected_answer": "Tapas",
    }
]
Pretty(handmade_eval_rows)

judge_model_id = "llama32"
JUDGE_PROMPT = """
Given a QUESTION and GENERATED_RESPONSE and EXPECTED_RESPONSE.

Compare the factual content of the GENERATED_RESPONSE with the EXPECTED_RESPONSE. Ignore any differences in style, grammar, or punctuation.
  The GENERATED_RESPONSE may either be a subset or superset of the EXPECTED_RESPONSE, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
  (A) The GENERATED_RESPONSE is a subset of the EXPECTED_RESPONSE and is fully consistent with it.
  (B) The GENERATED_RESPONSE is a superset of the EXPECTED_RESPONSE and is fully consistent with it.
  (C) The GENERATED_RESPONSE contains all the same details as the EXPECTED_RESPONSE.
  (D) There is a disagreement between the GENERATED_RESPONSE and the EXPECTED_RESPONSE.
  (E) The answers differ, but these differences don't matter from the perspective of factuality.

Give your answer in the format "Answer: One of ABCDE, Explanation: ".

Your actual task:

QUESTION: {input_query}
GENERATED_RESPONSE: {generated_answer}
EXPECTED_RESPONSE: {expected_answer}
"""

scoring_params = {
    "llm-as-judge::base": {
        "judge_model": judge_model_id,
        "prompt_template": JUDGE_PROMPT,
        "type": "llm_as_judge",
        "judge_score_regexes": ["Answer: (A|B|C|D|E)"],
    },
}

scoring_response = client.scoring.score(
    input_rows=handmade_eval_rows, scoring_functions=scoring_params
)
Pretty(scoring_response)

ScoringScoreResponse(
    results={
        'llm-as-judge::base': ScoringResult(
            aggregated_results={},
            score_rows=[
                {
                    'score': 'C',
                    'judge_feedback': 'Answer: C, Explanation: The GENERATED_RESPONSE contains all the same details
as the EXPECTED_RESPONSE.'
                },
                {
                    'score': 'C',
                    'judge_feedback': 'Answer: C, Explanation: The GENERATED_RESPONSE contains all the same details
as the EXPECTED_RESPONSE.'
                }
            ]
        )
    }
)

You should have gotten at least a C from the judge, and you can see the reasoning for it in the `judge_feedback` field.  
Feel free to try out some other inputs, generated answers, and expected answers 🧪

# Involve the LLM

So far we have just hardcoded the generated answers but these should be generated from an LLM, otherwise we are just evaluating our human selves.  
To do this, let's send some requests to our LLM through llamastack, and then also to our backend and see how that looks like.

In [9]:
model_id = "llama32"

model_eval_rows = [
    {
        "input_query": "What is your favorite Spanish food?",
        "expected_answer": "Tapas",
    },
    {
        "input_query": "What is your favorite Turkish food?",
        "expected_answer": "Baklava",
    }
]

Note how this time we don't have any generated answers yet, those will come from the LLM directly this time.

In [10]:
for eval_row in model_eval_rows:
    response = client.inference.chat_completion(
        model_id=model_id,
        messages=[
            {"role": "user", "content": eval_row["input_query"]}
        ]
    )
    eval_row["generated_answer"] = response.completion_message.content

Pretty(model_eval_rows)


[
    {
        'input_query': 'What is your favorite Spanish food?',
        'expected_answer': 'Tapas',
        'generated_answer': "I don't have personal preferences or taste buds, but I can suggest some popular and 
delicious Spanish dishes that you might enjoy:\n\n1. Paella: A classic Spanish dish made with saffron-infused rice,
vegetables, and your choice of protein (chicken, seafood, or chorizo).\n2. Tapas: Small, shareable plates of food, 
such as patatas bravas (spicy fried potatoes), tortilla española (Spanish omelette), and croquetas (deep-fried 
balls filled with ham, fish, or chicken).\n3. Gazpacho: A refreshing cold soup made from tomatoes, peppers, 
cucumbers, and bread, originating from the Andalusia region.\n4. Jamón ibérico: A cured ham from the Iberian 
Peninsula, known for its rich flavor and velvety texture.\n5. Churros con chocolate: Fried dough sticks coated in 
sugar, served with a rich and creamy chocolate dipping sauce.\n6. Empanada gallega: A savory pastry filled with 
seafood, meat, or vegetables, originating from the Galicia region.\n7. Tortilla de patatas: A thick, round omelette
made with potatoes, onions, and sometimes ham or chorizo.\n\nThese are just a few examples of the many delicious 
Spanish dishes out there. What's your favorite Spanish food?"
    },
    {
        'input_query': 'What is your favorite Turkish food?',
        'expected_answer': 'Baklava',
        'generated_answer': "I'm just a language model, I don't have personal preferences or taste buds, but I can 
tell you about some popular Turkish dishes that you might enjoy!\n\nSome of the most well-known and beloved Turkish
foods include:\n\n1. Doner Kebab: A classic Turkish dish made from layers of lamb or beef stacked on a vertical 
spit and served in a crispy sesame-topped bun with salad, vegetables, and sauce.\n2. Lahmacun (Turkish Pizza): A 
thin crust topped with minced meat, onions, and spices, often served with lemon juice and herbs.\n3. Borek: Flaky 
pastry filled with cheese, spinach, or minced meat, often served as a snack or appetizer.\n4. Manti (Turkish 
Ravioli): Steamed dumplings filled with meat and spices, served with yogurt and garlic sauce.\n5. Kofte: Meatballs 
made from ground meat (usually beef or lamb) and spices, often served with rice and vegetables.\n6. Sis Kebab: 
Skewers of marinated meat (usually lamb or beef) grilled over an open flame.\n7. Baklava: A sweet pastry made from 
layers of phyllo dough, nuts, and honey, often served as a dessert.\n\nThese are just a few examples of the 
delicious foods you can find in Turkey. Each region has its own specialties, so there's always something new to 
try!\n\nWhich one of these dishes sounds appealing to you?"
    }
]

And now we can just evaluate these answers just like we did before ✅

In [11]:
scoring_response = client.scoring.score(
    input_rows=model_eval_rows, scoring_functions=scoring_params
)
Pretty(scoring_response)

ScoringScoreResponse(
    results={
        'llm-as-judge::base': ScoringResult(
            aggregated_results={},
            score_rows=[
                {
                    'score': 'E',
                    'judge_feedback': 'Answer: E, Explanation: The GENERATED RESPONSE contains a list of popular 
Spanish dishes, including Tapas, but it does not express a personal preference or opinion, which is why the 
differences in style and the additional information do not affect the factual content.'
                },
                {
                    'score': 'E',
                    'judge_feedback': 'Answer: E, Explanation: The GENERATED RESPONSE contains a list of popular 
Turkish dishes, including Baklava, but it does not directly answer the question "What is your favorite Turkish 
food?" as it is phrased. The GENERATED RESPONSE is focused on providing information about Turkish cuisine, while 
the EXPECTED RESPONSE is a personal preference. The differences in the question and response do not affect the 
factual content, as the GENERATED RESPONSE provides accurate information about Turkish dishes, including Baklava.'
                }
            ]
        )
    }
)

# Evaluate the Backend

We can also evaluate our backend instead of the model, all we need to do is send the `input queries` to the backend and put the inputs into `generated_answer`.  
Since our backend is prompted to summarize text, we add tests that works well for such tasks.

In [12]:
backend_url = "http://canopy-backend.user1-canopy.svc.cluster.local:8000"
endpoint_to_test = "/summarize"

backend_eval_rows = [
    {
        "input_query": "Llama 3.2 is a state-of-the-art language model that excels in various natural language processing tasks, including summarization, translation, and question answering.",
        "expected_answer": "Llama 3.2 is a top-tier language model for NLP tasks.",
    },
    {
        "input_query": "Artificial intelligence and machine learning have revolutionized numerous industries in recent years. \
From healthcare diagnostics that can detect diseases earlier than human doctors, to autonomous vehicles that promise safer transportation, \
to recommendation systems that personalize our digital experiences, AI technologies are becoming increasingly sophisticated. \
However, these advances also bring challenges including ethical concerns about bias in algorithms, job displacement due to automation, and the need for robust data privacy protections?",
        "expected_answer": "AI and ML have transformed industries through healthcare diagnostics, autonomous vehicles, and recommendation systems, but also raise concerns about bias, job displacement, and privacy.",
    },
]

In [13]:
def send_request(payload, url):
    import httpx
    import json
    full_response = ""

    with httpx.Client(timeout=None) as client:
        with client.stream("POST", url, json=payload) as response:
            for line in response.iter_lines():
                if line.startswith("data: "):
                    try:
                        data = json.loads(line[len("data: "):])
                        full_response += data.get("delta", "")
                    except json.JSONDecodeError:
                        continue

    return full_response

def prompt_backend(prompt, backend_url, endpoint_to_test):
    from urllib.parse import urljoin
    url = urljoin(backend_url, endpoint_to_test)
    payload = {
        "prompt": prompt
    }
    return send_request(payload, url)

In [14]:
for eval_row in backend_eval_rows:
    eval_row["generated_answer"] = prompt_backend(eval_row["input_query"], backend_url, endpoint_to_test)

Pretty(backend_eval_rows)

ConnectError: [Errno -2] Name or service not known

And again, as soon as we have what we want to evaluate in a json format, we can evaluate it with Llamastack.

In [ ]:
scoring_response = client.scoring.score(
    input_rows=backend_eval_rows, scoring_functions=scoring_params
)
Pretty(scoring_response)

# Datasets 📖

Finally, let's use a dataset with already populated inputs and expected answers and see how well our model and backend does against those.

## SimpleQA

SimpleQA is a (as the name suggests) simple dataset with questions and answers that you can run on your model.  
You can find the full dataset, as well as a few others, here: https://huggingface.co/llamastack/datasets  
In our case, we don't want to wait to evaluate the full dataset, so we will take 5 examples from the training part of this dataset to test our model on.  
Similarily, we will just test on our model this time, partially because our `summarize` endpoint is not prompted for handling QA, and partially because we have already seen how we can evaluate our backend above.  

To fetch the dataset we use Llamastack again, where we can register the dataset which allows us to fetch data from it.

In [ ]:
simpleqa_dataset_id = "huggingface::simpleqa"

_ = client.datasets.register(
    purpose="eval/messages-answer",
    source={
        "type": "uri",
        "uri": "huggingface://datasets/llamastack/simpleqa?split=train",
    },
    dataset_id=simpleqa_dataset_id,
)

dataset_eval_rows = client.datasets.iterrows(
    dataset_id=simpleqa_dataset_id,
    limit=5,
)

In [ ]:
Pretty(dataset_eval_rows)

As you can see, the dataset is already formated with `input_query` and `expected_answer`, and we get some extra information such as `metadata` and `chat_completion_input`.  

We could now just input this evaluation set to our model just like we did before, but since we are only evaluating the model we will make use of another Llamastack functionality called `benchmarks`.  
This simply passes the dataset through the model and returns the response.

In [ ]:
client.benchmarks.register(
    benchmark_id="meta-reference::simpleqa",
    dataset_id=simpleqa_dataset_id,
    scoring_functions=["llm-as-judge::base"],
)

response = client.eval.evaluate_rows(
    benchmark_id="meta-reference::simpleqa",
    input_rows=dataset_eval_rows.data,
    scoring_functions=["llm-as-judge::base"],
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": model_id,
            "sampling_params": {
                "strategy": {
                    "type": "greedy",
                },
                "max_tokens": 4096,
                "repeat_penalty": 1.0,
            },
        },
    },
)
Pretty(response)


To summarize, we have now used the Llamastack Eval endpoint to evaluate our raw LLM as well as our backend system, both with custom evaluations and with those fetched from a dataset.  
With this knowledge, we can now build an evaluation workflow that lets us test our backend and model before it goes into production 👏